#### Aplicamos algunas transformaciones a los datos de cada plataforma de manera que los datos sean consistentes a la hora de generar con ellos un modelo de aprendizaje 


In [1]:
import pandas as pd

import numpy as np

#funcion para obtener los datasets de google drive
from  raw_data.conexiones_lake import obtener_datos_plataforma

Dataset amazon

La funcion obtener_datos_plataforma obtiene los datos crudos de google drive


In [119]:
amazon = obtener_datos_plataforma('amazon')

In [92]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


country y date_added presentan muchos valores nulos, son candidatos a eliminarse pues date_added tiene alrededor del 1% de datos no nulos y country alrededor del 6% completos

Los incompletos de director y cast son campos inapropiados para imputación, pues una película es puro arte y su identidad está definida fuertemente por el valor de estos campos. Optamos por ignorar los nulos por el momento.

In [120]:
amazon.drop(['date_added','country'],axis=1,inplace=True,errors='ignore')

veamos los tipos de duración

In [114]:
dur = amazon.duration.dropna().apply(lambda x : x.split(' ')[1])

In [103]:
dur.unique()

array(['min', 'Season', 'Seasons'], dtype=object)

A la hora de separar tipo de duración elegimos dominio: min y season, por lo tanto seasons debe ser convertido a season 

Dataset disney

In [123]:
disney = obtener_datos_plataforma('disney')

In [105]:
disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


En este caso los nulos de campo director son menor al 25%, mantendremos ese campo

Dataset hulu

In [106]:
hulu = obtener_datos_plataforma('hulu')

In [107]:
hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


cast y director presentan muchos valores nulos, son candidatos a eliminarse pues date_added tiene alrededor del 1% de datos no nulos y country alrededor del 6% completos

In [108]:
hulu.drop(['cast','director'],axis=1,inplace=True,errors='ignore')

Dataset netflix

In [109]:
netflix = obtener_datos_plataforma('netflix')

In [110]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


El porcentaje de nulos es poco, se conservan esas columnas para posible imputación

Generamos campo id con prefijo dado por nombre de plataforma

In [2]:
def generar_id(prefijo,df):
  """
  Concatena el valor de prefijo a la columna show_id del df.

  Parameters
  ---------
  prefijo: str
    letra que se añade al inicio de cada valor del campo show_id
  df: object
    dataframe con la columna show_id
  """

  df["id"] = prefijo+df['show_id']  
  
  # eliminamos columna show_id pues no es necesaria con el nuevo id
  df.drop(columns='show_id',index=1,inplace=True)

Reemplazamos valores nulos de rating

In [3]:
def completar_rating(df):
  """
  Completa los nulos del campo rating con letra G.

  Parameters
  ---------
   df: object
    dataframe con la columna rating
  """

  df['rating'].fillna('G',inplace=True)

Formateamos fechas

In [117]:
def formatear_fecha(df):
  """
  Formatea fechas a formato AAA-mm-dd del df.

  Parameters
  ----------
  df: object
    dataframe con la columna que represente fechas
  """
  if 'date_added' in df.columns.values:
    df['date_added'] = pd.to_datetime(df.date_added)

Campos de texto a minusculas

In [51]:
def texto_en_minuscula(df):
  """
  Convierte todos los campos del df de texto en minúsculas.

  Parameters
  ---------
  df: object
    dataframe con columnas de texto
  """

  # seleccionamos las columnas de texto
  columns = np.array( df.columns)

  # excluimos las de fechas
  columns = [column for column in columns if column != 'date_added' and column != 'release_year']
  
  #solo las seleccionadas van a lowercase
  df[columns] = df[columns].fillna('').apply(lambda val: val.str.lower())  


Separamos campo duration en duration_int y duration_type

In [63]:
def obtener_valor_tipo(item,posicion):
   
  if isinstance(item,str):
  
    if ' ' in item:
  
      num = item.split()[posicion]            
  
      return num.strip()

  if posicion == 0:
    return 0
  else:
    return ''

In [81]:
def separar_duration(df):
  """
  Separa el campo duration en parte entera y tipo de duración

  Parameters
  ---------
  df: object
    dataframe con columna duration
  """

  df['duration_int'] = df['duration'].apply(obtener_valor_tipo,args=(0,))

  df['duration_int'] = df['duration_int'].astype('float')
 
  df['duration_int'] = df['duration_int'].astype('Int64')
 
  df['duration_type'] = df['duration'].apply(obtener_valor_tipo,args=(1,))
  
  #hay valores con error sintactico
  df['duration_type'].replace('seasons','season', inplace=True)

  # eliminamos columna duration que ha sido separada y no es necesaria
  df.drop(columns='duration', axis=1, inplace=True, errors='ignore')

Agrupamos las funciones en una sola

In [20]:
def transformar(prefijo,df):
  """
  Aplica transformaciones al df

  Parameters
  ---------
  df: object
    dataframe a transformar
  prefijo: str
    letra que se añade al inicio de cada valor del campo show_id
  """

  generar_id(prefijo, df)

  completar_rating(df)    

  formatear_fecha(df)

  texto_en_minuscula(df)

  separar_duration(df)

  # reordenamos las columnas para que id este primer 
  columns = np.array(df.columns)

  columns = [column for column in columns if column != 'id']

  df = df[['id']+columns]

  return df

Transformamos los datasets

In [121]:
amazon_df = transformar('a',amazon)

In [124]:
disney_df = transformar('d',disney)

In [25]:
hulu_df = transformar('h',hulu)

In [86]:
netflix_df = transformar('n',netflix)

Exportamos las transformaciones

In [28]:
amazon_df.to_csv('../transformaciones/amazon.csv',index=False)

In [29]:
disney_df.to_csv('../transformaciones/disney.csv',index=False)

In [30]:
hulu_df.to_csv('../transformaciones/hulu.csv',index=False)

In [88]:
netflix_df.to_csv('../transformaciones/netflix.csv',index=False)